In [1]:
# after downloading the data file, we import the data in MySQL workbench directly 
# Step 1: create a schema named dbms having a table named forestdata
# step 2: 'table data import wizard' to import data from csv to DB 

In [ ]:
!pip install pandas mysql-connector-python

In [3]:
# import libraries
# mysql-connector-python allows Python to connect to a MySQL database, execute queries and retrieve data
import mysql.connector
from mysql.connector import Error
import pandas as pd
import random as rd
import math
import numpy as np

from itertools import chain, combinations

In [ ]:
masterTable = pd.read_csv('data.csv')
masterTable.head()

In [ ]:
dataset = masterTable.iloc[: , :10]
dataset.head()

In [6]:
def generate_subsets(arr):
    all_subsets = []
    for r in range(1, len(arr) + 1):
        subsets_r = list(combinations(arr, r))
        all_subsets.extend(subsets_r)

    return [list(subset) for subset in all_subsets]

In [7]:
def genr_uniform_distr(col,count):
  l_uni = []
  col = col.to_list()
  min_val = min(col)
  max_val = max(col)
  for i in range(count):
    l_uni.append(rd.randint(min_val,max_val))
  return l_uni

def genr_data_distr(col,count):
  l_data = []
  col =  col.to_list()
  for i in range(count):
    index = rd.choice(col)
    l_data.append(index)
  return l_data


In [8]:
def uni_widths(col,query_centre):
  col = col.to_list()
  min_val = min(col)
  max_val = max(col)
  lower_bound = 0
  if(abs(query_centre - min_val) < abs(query_centre - max_val) ):
    upper_bound = abs(query_centre - min_val)
  else:
    upper_bound = abs(query_centre - max_val)
  #print(min_val,max_val,upper_bound)
  width = np.random.uniform(0, upper_bound, 1)
  return math.ceil(width[0])

In [9]:
def expo_width(col,query_centre):
  col = col.to_list()
  min_val = min(col)
  max_val = max(col)
  lower_bound = 0
  if(abs(query_centre - min_val) < abs(query_centre - max_val) ):
    upper_bound = abs(query_centre - min_val)
  else:
    upper_bound = abs(query_centre - max_val)
  #print(min_val,max_val,upper_bound)
  scale = (upper_bound) // 4
  width = np.random.exponential(scale)
  return math.ceil(width)

In [10]:
def find_bounds(col,n):
  l_bounds = []
  if(n%2 ==0):
    n_uniform = n_data = n//2
  else:
    n_uniform = n//2
    n_data = n_uniform + 1
  #col = col.to_list()
  uniform_list = genr_uniform_distr(col,n_uniform)
  data_list = genr_uniform_distr(col,n_data)
  for i in range(n_uniform):
    query_centre = uniform_list[i]
    width = uni_widths(col,query_centre)
    lb = query_centre - width
    ub = query_centre + width
    bounds = [lb,ub]
    l_bounds.append(bounds)
  for i in range(n_data):
    query_centre = data_list[i]
    width = uni_widths(col,query_centre)
    lb = query_centre - width
    ub = query_centre + width
    bounds = [lb,ub]
    l_bounds.append(bounds)
  return l_bounds

In [11]:
def getRanges(cols, noSamplesSet):
  attToDist = dict()
  for c in cols:
    attToDist[c] = find_bounds(dataset[c], noSamplesSet)
  return attToDist

In [12]:
def create_zero_dataframe(n, D):
    data = {f"Attribute_{i}": [0.1] * n for i in range(1, D+1)}
    df = pd.DataFrame(data)
    return df

def generateSkeletonData(n, columns):
  zero_df = create_zero_dataframe(n, 2*len(columns))
  newColNames =[]
  for c in columns:
    newColNames.append(c + " LB")
    newColNames.append(c + " UB")
  zero_df.columns = newColNames
  return zero_df

In [13]:
def findIndex(col, columns):
  for i,c in enumerate(columns):
    if (c == col):
      return i

In [14]:
n = 20000
columns = list(dataset.columns)
subsets = generate_subsets(columns)

sampleSize = math.ceil(n/len(subsets))
distributions = getRanges(columns, sampleSize)
data = generateSkeletonData(sampleSize*len(subsets), columns)
ctr = 0
for s in subsets:
  for c in s:
    index = findIndex(c, columns)
    data.iloc[ctr : ctr+sampleSize, 2*index : 2*index+2] = distributions[c]
  ctr = ctr + sampleSize


In [ ]:
data

In [17]:
import mysql.connector
from mysql.connector import Error

# to establish the connection- host, username(usually root), sql workbench password and database name are required 
try:
        connection = mysql.connector.connect(
                                         host='localhost',
                                         database='dbms',
                                         user='root',
                                         password='Tol@sql')
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)

except Error as err:
        print("Error while connecting to MySQL", err)
        
#     finally:
#         if (connection.is_connected()):
#             cursor.close()
#             connection.close()
#             print("MySQL connection is closed")

Connected to MySQL Server version  8.0.36
You're connected to database:  ('dbms',)


In [18]:
connection

In [19]:
def read_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f'Error: {err}')

In [ ]:
selection = """
select * from forest
"""

results = read_query(connection, selection)
for result in results:
    print(result)

In [20]:
cursor = connection.cursor()

In [21]:
def generate_execute_queries(dataframe, table_name):
    query_results = []
    
    # Get the total number of rows in the table for calculating proportions
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    total_rows = cursor.fetchone()[0]
    
    for index, row in dataframe.iterrows():
        where_conditions = []
        for column in dataframe.columns:
            if 'LB' in column:
                ub_column = column.replace('LB', 'UB')
                if row[column] != 0.1 or row[ub_column] != 0.1:
                    condition = f"{column[:-3]} BETWEEN {row[column]} AND {row[ub_column]}"
                    where_conditions.append(condition)

        where_clause = ' AND '.join(where_conditions)
        
        if where_clause:  # Ensure there's a condition to avoid empty WHERE clause
            sql_query = f"SELECT COUNT(*) FROM {table_name} WHERE {where_clause}"
            cursor.execute(sql_query)
            count = cursor.fetchone()[0]
            proportion = count / total_rows  # Calculate the proportion
            query_results.append(proportion)  # Append only the proportion to results
        else:
            query_results.append(0)  # Append 0 if no conditions are met

    return query_results

# Assuming 'data' is your DataFrame and 'forest' is your table name
proportions = generate_execute_queries(data, 'forest')

# Close the database connection
# conn.close()

# Convert proportions to a DataFrame
results_df = pd.DataFrame(proportions, columns=['Proportion'])

In [31]:
pd.DataFrame(proportions, columns=['Proportion'])

,Proportion
0,0.355910
1,0.000043
2,0.986703
3,0.043276
4,0.007934
...,...
20455,0.000000
20456,0.000000
20457,0.000000
20458,0.000000


In [78]:
pd.set_option('display.max_rows', None)
selectivity = pd.DataFrame(proportions, columns=['Proportion'])
selectivity

,Proportion
0,0.355910
1,0.000043
2,0.986703
3,0.043276
4,0.007934
5,0.188803
6,0.001391
7,0.003509
8,0.869655
9,0.130261


In [ ]:
selectivity.to_csv('selectivity.csv', index=False)

# TESTING:

In [ ]:
pd.DataFrame(proportions, columns=['Proportion'])

In [70]:
specific_row = data.loc[12657]
specific_row 

In [ ]:
specific_row = data.loc[12657]

where_conditions = []

for column in data.columns:
    if 'LB' in column:
        ub_column = column.replace('LB', 'UB')
        if specific_row[column] != 0.1 or specific_row[ub_column] != 0.1:
            condition = f"{column[:-3]} BETWEEN {specific_row[column]} AND {specific_row[ub_column]}"
            where_conditions.append(condition)

where_clause = ' AND '.join(where_conditions)

table_name = 'forest'  
if where_clause:  
    sql_query = f"SELECT COUNT(*) FROM {table_name} WHERE {where_clause}"
else:
    sql_query = f"SELECT COUNT(*) FROM {table_name}"  

print(sql_query)


In [ ]:
cursor.execute(f"SELECT COUNT(*) FROM forest")
total_rows = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM forest WHERE Vertical_Distance_To_Hydrology BETWEEN 523.0 AND 531.0 AND Horizontal_Distance_To_Roadways BETWEEN 4620.0 AND 4744.0 AND Hillshade_9am BETWEEN 12.0 AND 56.0 AND Hillshade_Noon BETWEEN 154.0 AND 234.0 AND Horizontal_Distance_To_Fire_Points BETWEEN 6928.0 AND 7078.0')
count = cursor.fetchone()[0]

count/total_rows